# The Evolution of the 3-Point Shot in the NBA 
## Introduction
In 1980, the National Basketball Association (NBA) introduced the 3-point line. At the time it was not considered significant, however over time we have seen a drastic increase on the reliance of the three pointer, and today is considered an important part of any team's gameplan. Through their dominance in the mid to late 2010s, the Golden State Warriors have been recognized as having revolutionized the game and making the three pointer a must-have attribute. The logic is simple: a three-point shot is worth more than a 2-point shot, so if a team can make enough 3-point shots, this will outweigh the 2-pointers. However, there have been questions raised about the effectiveness of shooting many 3 pointers, and if it truly translates into a winnning strategy. 

In this analysis, I will evaluate the increase in three point shots taken since the introduction of the 3-point line, and how/if taking more three-pointers translates into more victories. Some questions that will be answered:

    - Does making/shooting more 3-pointers translate into more wins? 
    - How efficient (in terms of shot-making percentage) do you have to be to make a 3-pointer more valuable than a 2-pointer? 
    - How does the league compare now vs. 40 years ago? More scoring? Do better teams make more 3-pointers? 

##  Gathering Data 
All of the libraries needed for this tutorial to scrape, tidy and visualize the data are imported here. 

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn
from bs4 import BeautifulSoup
import requests

## Source for Data
I scraped data for each season since 1980 from [Basketball Reference](basketball-reference.com). Unfortunately, the table there are not static, but fortunately they had an embedded link for the table, which I was able to use to scrape the tables I wanted! The tables were stored by season, so I just gathered each team's total stats for the season and looped through each link to get every season from 1980-2021 total stats for each team. 




In [9]:
year = 1980
stats_url = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fleagues%2FNBA_{year}.html&div=div_team-stats-base'
standings_url = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fleagues%2FNBA_{year}.html&div=div_misc_stats'
df = None 
for year in range(1980, 2022):
    if df is None: 
        df = pd.read_html(stats_url)[0]
        df['Season'] = year 
        
    else: 
        temp = pd.read_html(stats_url)[0]
        temp['Season'] = year 
        df = pd.concat([df, temp])
df = df[['Team', 'G', 'FG', 'FGA', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'PTS', 'Season']]
df



,Team,G,FG,FGA,3P,3PA,Season
0,San Antonio Spurs*,82,3856,7738,52,206,1980
1,Los Angeles Lakers*,82,3898,7368,20,100,1980
2,Cleveland Cavaliers,82,3811,8041,36,187,1980
3,New York Knicks,82,3802,7672,42,191,1980
4,Boston Celtics*,82,3617,7387,162,422,1980
...,...,...,...,...,...,...,...
18,Atlanta Hawks*,82,3261,7027,13,75,2021
19,Golden State Warriors,82,3527,7318,27,121,2021
20,Portland Trail Blazers*,82,3408,7167,26,132,2021
21,Utah Jazz,82,3382,6817,59,185,2021
